In [1]:
#!/usr/bin/python3
import datetime
# 存取當下的日期(年月日)
datetime.datetime.now()
now_year, now_month, now_day = str(datetime.datetime.now()).split(' ')[0].split('-')
timestamp_now = int(datetime.datetime(int(now_year), int(now_month), int(now_day), 0, 0).timestamp())

#now_year, now_month, now_day

In [2]:
# 製造該ETF當下的網址(from yahoo finance)
def url_of_ETF_now(etf_name):
    url='https://finance.yahoo.com/quote/ETF_NAME/history?period1=1448899200&period2=NOW_TIME&interval=1d&filter=history&frequency=1d.html'
   
    exact_url=url.replace('ETF_NAME', str(etf_name))
    exact_url=exact_url.replace('NOW_TIME', str(timestamp_now))
    
    return exact_url
#url_of_ETF_now('VCIT')

In [3]:
# 讀標的ETF的名子
import csv

goal_ETF = []
with open('goal_ETF.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        #print(row[0])
        goal_ETF.append(row[0])
print(goal_ETF)

['LQD', 'VCSH', 'VCIT', 'IGSB', 'IGIB', 'SPSB', 'SPIB', 'USIG', 'VCLT', 'GSY', 'SLQD', 'ICSH', 'CORP', 'IGLB', 'SPLB', 'IGHG', 'LQDH', 'QLTA', 'PFIG', 'SKOR', 'IGBH', 'CBND', 'LKOR', 'LDRI', 'IGIH']


In [4]:
import requests

# 使用 requests.get 把網頁內容載下來
def get_response(etf_name):
    response = requests.get(url_of_ETF_now(etf_name))
    #response.text  # 展示載來的資料
    # 使用 split 這個函數 把文字做切割成不同行
    lines = response.text.split('{"date":')
    return lines

#get_response('LQD')
# 觀察切割後想要的資料的格式 及長度
#print(lines[1])
#print(lines[100])
#print(lines[300].split('}')[0])
#len(lines[300].split('}')[0].split(','))

In [5]:
def get_price(etf_name):    #取得該ETF各日價格
    
    newlines = []     # 定義一個我真的想要的那些行
    price = []        # 收集價格
    
    for line in get_response(etf_name):
        line = line.split('}')[0]             # 取'}'前的部分
        if len(line.split(',')) == 7:        # 用「,」切欄位 看是否為7欄

            newlines.append(line)          # 將 line 加到 newlines 中

    for i in range(len(newlines)-1):
        if newlines[i].split(',')[0].isdigit():     # 檢查非空白 
            
            adj_price = round(float(newlines[i].split(',')[6].split(':')[1][:-1]),2)
            price.append(adj_price)
            
    return price
#len(get_price('SPIB'))

In [6]:
#!/usr/bin/python3
# 檢查各個ETF價格筆數是否相同 順便存下筆數及價格
from statistics import mode
day_numbers = []
ETF_PRICE = {}
ETF_LEN = {}

for ETF in goal_ETF:
    P = get_price(ETF)
    L = len(P)
    while L == 0:            # 偶而可能是連線問題出錯 要重新取得
        P = get_price(ETF)
        L = len(P)

    day_numbers.append(L)
    ETF_PRICE.update({ETF: P})
    ETF_LEN.update({ETF: L})
    
    #print(L)

In [7]:
# print 出有缺資料的ETF
for ETF in ETF_LEN:
    if ETF_LEN.get(ETF) != mode(day_numbers):
        print(ETF)

SPSB
IGLB
IGBH


In [8]:
# 去除資料不足 (在2015/12/1後有出現沒資料的 有3個)
goal_ETF.remove('IGLB')
goal_ETF.remove('IGBH')
goal_ETF.remove('SPSB')
#goal_ETF

In [9]:
# 存取扣除缺漏後的價格資料
Price_Total = {}

for ETF in goal_ETF:
    Price_Total[ETF] = ETF_PRICE.get(ETF)

# 存取總日期資料


def Total_date(etf_name):
    date_Total = []
    lines = []
    
    for line in get_response(etf_name):
        line = line.split('}')[0]
        if len(line.split(',')) == 7:   
            lines.append(line) 
        
    for i in range(len(lines)-1):
        if lines[i].split(',')[0].isdigit():
            epoch_date = int(lines[i].split(',')[0])
            date_Total.append(str(datetime.datetime.fromtimestamp(epoch_date)).split(' ')[0])
    return date_Total
        

#Total_date(goal_ETF[0])

In [10]:
import pandas
from collections import OrderedDict

Total = OrderedDict(Price_Total)  
Total.update({'Date': Total_date(goal_ETF[0])})    # 加入日期
Total.move_to_end('Date',last = False)    # 把日期放最前面

df = pandas.DataFrame(Total)    # 輸出資料框架
df.head(20)

,Date,LQD,VCSH,VCIT,IGSB,IGIB,SPIB,USIG,VCLT,GSY,...,SPLB,IGHG,LQDH,QLTA,PFIG,SKOR,CBND,LKOR,LDRI,IGIH
0,2019-03-22,117.78,79.47,86.20,52.60,54.66,33.99,54.73,89.62,50.20,...,26.56,73.96,93.51,51.65,25.24,50.29,31.51,50.99,24.87,23.24
1,2019-03-20,117.61,79.48,86.18,52.59,54.63,33.98,54.66,89.48,50.20,...,26.53,73.92,93.47,51.63,25.26,50.10,31.53,50.44,24.87,23.18
2,2019-03-19,116.82,79.26,85.68,52.51,54.37,33.84,54.40,88.74,50.19,...,26.28,74.04,93.52,51.37,25.13,50.09,31.40,50.30,24.88,23.18
3,2019-03-18,116.80,79.24,85.65,52.49,54.36,33.83,54.40,88.68,50.17,...,26.31,73.92,93.36,51.36,25.14,50.19,31.41,50.46,24.80,23.19
4,2019-03-15,116.86,79.27,85.71,52.00,54.35,33.82,54.41,88.74,50.29,...,26.30,73.84,93.32,51.37,25.18,50.19,31.36,50.43,24.83,23.18
5,2019-03-14,116.44,79.17,85.43,52.44,54.20,33.77,54.26,88.29,50.30,...,26.15,74.16,93.36,51.24,25.14,50.11,31.31,50.30,24.93,23.09
6,2019-03-13,116.67,79.20,85.00,52.41,54.23,33.78,54.32,88.58,50.28,...,26.27,73.84,93.34,51.31,25.15,50.11,31.33,50.48,24.93,23.16
7,2019-03-12,116.86,79.16,85.56,52.44,54.21,33.80,54.40,88.83,50.27,...,26.36,73.78,93.00,51.38,25.18,50.16,31.32,50.48,24.88,23.13
8,2019-03-11,116.54,79.11,85.37,52.38,54.12,33.76,54.20,88.39,50.28,...,26.22,73.73,93.43,51.23,25.11,49.94,31.24,50.19,24.88,23.14
9,2019-03-08,116.41,79.10,85.28,52.33,54.02,33.73,54.19,88.28,50.26,...,26.18,73.86,93.20,51.21,25.13,49.94,31.24,50.03,24.92,23.14


In [11]:
# 輸出 IGLB IGBH 的部分 他們價格數目相同

P_T_IGLB_IGBH = {}
for ETF in ['IGLB', 'IGBH']:                           # 導入價格資料
    P_T_IGLB_IGBH[ETF] = get_price(ETF)

Total_IGLB_IGBH = OrderedDict(P_T_IGLB_IGBH)
Total_IGLB_IGBH.update({'Date': Total_date('IGLB')})   # 加入日期
Total_IGLB_IGBH.move_to_end('Date',last = False)       # 把日期放最前面

df = pandas.DataFrame(Total_IGLB_IGBH)
df.head(20)

,Date,IGLB,IGBH
0,2019-03-22,58.96,24.59
1,2019-03-20,58.81,24.60
2,2019-03-19,58.42,24.67
3,2019-03-18,58.46,24.67
4,2019-03-15,58.48,24.65
5,2019-03-14,58.23,24.65
6,2019-03-13,58.42,24.66
7,2019-03-12,58.62,24.76
8,2019-03-11,58.32,24.71
9,2019-03-08,58.18,24.59


In [12]:
# 輸出 SPSB 的部分 

P_T_SPSB = {}
P_T_SPSB['SPSB'] = get_price('SPSB')

Total_SPSB = OrderedDict(P_T_SPSB)
Total_SPSB.update({'Date': Total_date('SPSB')})
Total_SPSB.move_to_end('Date',last = False)

df = pandas.DataFrame(Total_SPSB)
df.head(20)

,Date,SPSB
0,2019-03-22,30.47
1,2019-03-20,30.48
2,2019-03-19,30.43
3,2019-03-18,30.42
4,2019-03-15,30.43
5,2019-03-14,30.41
6,2019-03-13,30.41
7,2019-03-12,30.42
8,2019-03-11,30.40
9,2019-03-08,30.38
